# Part 2: Finding Optimal Anchor Configurations


If you know about the RetinaNet architecture, you probably will be able to deduce that the anchor box generator is not generating anchor boxes with a high enough IoU with the spectogram targets to have the boxes land in the training target.
This is apparent, as in part 1 you can see that the oblong boxes slip through the cracks.  In order to remedy this problem, lets find an optimal anchor box configuration.

Unforatunetly, spectograms have little in common with typical images.  The size and aspect ratios of signals in a spectogram has nothing to do with the distribution of objects in the real world.  As such, we have to put a great deal of effort into finding optimal anchor configurations.

Lets explore our dataset to better understand the issue

In [2]:
import em_loader

In [ ]:
dataset, dataset_info = em_loader.load(
    split="train",
    bounding_box_format="xywh",
    batch_size=9,
    version=2,
)

# Data Understanding

## Aspect Ratios

In [ ]:
import numpy as np
from tqdm import tqdm
result = []

for batch in tqdm(iter(dataset)):
    boxes = batch[1]
    aspect_ratios = boxes[:, :, 2] / boxes [:, :, 3]
    aspect_ratios = aspect_ratios.to_tensor(-1.0)
    aspect_ratios = tf.reshape(aspect_ratios, (-1))
    aspect_ratios = tf.gather(aspect_ratios, tf.where(aspect_ratios != -1.0))
    result.append(aspect_ratios.numpy())
result = np.concatenate(result)
aspect_ratios = result

In [ ]:
import numpy as np
from tqdm import tqdm
result = []
for batch in tqdm(iter(train_ds)):
    boxes = batch[1]
    aspect_ratios = boxes[:, :, 2] / boxes [:, :, 3]
    aspect_ratios = aspect_ratios.to_tensor(-1.0)
    aspect_ratios = tf.reshape(aspect_ratios, (-1))
    aspect_ratios = tf.gather(aspect_ratios, tf.where(aspect_ratios != -1.0))
    result.append(aspect_ratios.numpy())
result = np.concatenate(result)
aspect_ratios = result

import seaborn as sns
sns.histplot(aspect_ratios, bins=15)

## Area

In [ ]:

distribution, threshold = np.histogram(aspect_ratios, bins=20)

total = distribution.sum()
count = 0
previous = 0
for d, t in zip(distribution, threshold):
    print(f'{(d/total) * 100:.2f}% between {previous:.9f}-{t:.9f}')
    previous = t

import numpy as np
from tqdm import tqdm
result = []
for batch in tqdm(iter(train_ds)):
    boxes = batch[1]
    area = boxes[:, :, 2] * boxes [:, :, 3]
    area = area.to_tensor(-1.0)
    area = tf.reshape(area, (-1))
    side_lengths = tf.math.sqrt(tf.gather(area, tf.where(area != -1.0)))
    result.append(side_lengths.numpy())
side_lengths = np.concatenate(result)

import seaborn as sns
sns.histplot(side_lengths, bins=15)

distribution, threshold = np.histogram(side_lengths, bins=40)

total = distribution.sum()
count = 0
previous = 0
for d, t in zip(distribution, threshold):
    print(f'{(d/total) * 100:.2f}% between {previous:.9f}-{t:.9f}')
    previous = t